# Does the mask-or-vaccine requirement affect public attendance at businesses? Or even their ratings?

between Florida and New York are there differences between rating or review percentage differences
would take the sums of the current day minus the sums of the previous day divided by the sums of the previous day ?

In [2]:
import pandas as pd
import numpy as np
import random
from collections import Counter
from pathlib import Path
import os
from statsmodels.stats.api import CompareMeans, DescrStatsW
pd.options.display.float_format = '{:,.4f}'.format

In [3]:
cg_df: pd.DataFrame = pd.read_parquet('cg_est', engine='pyarrow')
bh_df: pd.DataFrame = pd.read_parquet('bus_holdings', engine='pyarrow')
bus_cats_df: pd.DataFrame = pd.read_parquet('bus_cats', engine='pyarrow')

In [4]:
cg_df

,CountyName,EstimationYear,EstimatedPopulation,lastpop,relative_delta,abs_delta,StateName
0,Mobile County,2019,413210,413908,-0.0017,-698,Alabama
1,Cherokee County,2019,26196,26014,0.0070,182,Alabama
2,Hale County,2019,14651,14749,-0.0066,-98,Alabama
3,Jefferson County,2019,658573,659429,-0.0013,-856,Alabama
4,Lawrence County,2019,32924,32941,-0.0005,-17,Alabama
...,...,...,...,...,...,...,...
3215,Washakie County,2019,7805,7877,-0.0091,-72,Wyoming
3216,Carbon County,2019,14800,14879,-0.0053,-79,Wyoming
3217,Crook County,2019,7584,7445,0.0187,139,Wyoming
3218,Weston County,2019,6927,6924,0.0004,3,Wyoming


In [5]:
bh_df

,BusinessName,ChainName,BusinessRating,ReviewCount,previous_review_cnt,previous_rating,abs_review_diff,abs_rating_diff,total_review_cnt_delta,total_bus_rating_delta,CloseDate
0,1-chinese-restaurant-coinjock,1 Chinese Restaurant,3.000000000,2,NaN,None,NaN,None,0.0000,0E-9,2021-12-29
1,1-cow-ten-sleep,1 Cow,4.500000000,6,NaN,None,NaN,None,0.0000,0E-9,2021-12-29
2,1-more-bar-and-grille-bright,1 More Bar & Grille,4.000000000,9,NaN,None,NaN,None,0.0000,0E-9,2021-12-29
3,1-north-steakhouse-hampton-bays-2,1 North Steakhouse,4.000000000,147,NaN,None,NaN,None,2.0000,0E-9,2021-12-29
4,10-40-coffee-west-plains,10/40 Coffee,5.000000000,11,NaN,None,NaN,None,0.0000,0E-9,2021-12-29
...,...,...,...,...,...,...,...,...,...,...,...
1982530,zzaam-hamden-hamden,ZZAAM Hamden,4.500000000,55,55.0000,4.500000000,0.0000,0E-9,1.0000,0E-9,2022-02-09
1982531,zzs-sports-bar-and-grill-adrian,ZZ's Sports Bar & Grill,3.500000000,10,10.0000,3.500000000,0.0000,0E-9,0.0000,0E-9,2022-02-09
1982532,zzzz-s-pizzzza-burlington,ZZZZ’s PIzzzza,5.000000000,1,1.0000,5.000000000,0.0000,0E-9,0.0000,0E-9,2022-02-09
1982533,àlavita-boise-2,ÀLAVITA,4.000000000,307,307.0000,4.000000000,0.0000,0E-9,3.0000,0E-9,2022-02-09


In [6]:
bus_cats_df

,BusinessKey,BusinessName,ChainName,PaymentLevelName,Longitude,Latitude,BusinessCategoryName,CityName,CountyName,CountryName,StateName
0,1,the-pie-source-new-mexico,The Pie Source,Unknown,-108.128867000,34.298479000,Bakeries,New Mexico,Catron County,GB,ABE
1,1,the-pie-source-new-mexico,The Pie Source,Unknown,-108.128867000,34.298479000,Coffee & Tea,New Mexico,Catron County,GB,ABE
2,1,the-pie-source-new-mexico,The Pie Source,Unknown,-108.128867000,34.298479000,Desserts,New Mexico,Catron County,GB,ABE
3,6,pizza-hut-geneva-2,Pizza Hut,Very Low,-85.887118000,31.049032000,Pizza,Geneva,Geneva County,US,Alabama
4,6,pizza-hut-geneva-2,Pizza Hut,Very Low,-85.887118000,31.049032000,Chicken Wings,Geneva,Geneva County,US,Alabama
...,...,...,...,...,...,...,...,...,...,...,...
126522,63308,taco-johns-thermopolis,Taco John's,Unknown,-108.209207000,43.650328000,Fast Food,Thermopolis,Hot Springs County,US,Wyoming
126523,63316,the-crown-inn-london,The Crown Inn,Unknown,0.073200000,51.410740000,Pubs,London,Laurel County,GB,XGL
126524,63317,boreal-kennels-carcross,Boreal kennels,Unknown,-134.704570000,60.169670000,Fishing,Carcross,Skagway Municipality,CA,YT
126525,63317,boreal-kennels-carcross,Boreal kennels,Unknown,-134.704570000,60.169670000,Tours,Carcross,Skagway Municipality,CA,YT


In [7]:
florida_newyork = cg_df.loc[cg_df['StateName'].isin(values=['Florida', 'New York'])]
florida_newyork

,CountyName,EstimationYear,EstimatedPopulation,lastpop,relative_delta,abs_delta,StateName
320,Alachua County,2019,269043,268851,0.0007,192,Florida
321,Hillsborough County,2019,1471968,1451919,0.0138,20049,Florida
322,Miami-Dade County,2019,2716940,2714854,0.0008,2086,Florida
323,Polk County,2019,724777,706597,0.0257,18180,Florida
324,Marion County,2019,365579,359062,0.0182,6517,Florida
...,...,...,...,...,...,...,...
1885,Chenango County,2019,47207,47445,-0.0050,-238,New York
1886,Suffolk County,2019,1476601,1480830,-0.0029,-4229,New York
1887,Saratoga County,2019,229863,230170,-0.0013,-307,New York
1888,Schenectady County,2019,155299,155079,0.0014,220,New York


In [8]:
florida_newyork_pop_sums = florida_newyork.groupby(['StateName'], as_index=False)['EstimatedPopulation'].sum()
florida_newyork_pop_sums.sort_values(by='EstimatedPopulation', ascending=False).head()

,StateName,EstimatedPopulation
9,Florida,21477737
32,New York,19453561
0,Alabama,0
39,Puerto Rico,0
29,New Hampshire,0


In [9]:
florida_pop = florida_newyork_pop_sums.loc[florida_newyork_pop_sums.StateName == 'Florida', 'EstimatedPopulation']
florida_pop

9    21477737
Name: EstimatedPopulation, dtype: int64

In [10]:

newyork_pop = florida_newyork_pop_sums.loc[florida_newyork_pop_sums.StateName == 'New York', 'EstimatedPopulation']
newyork_pop

32    19453561
Name: EstimatedPopulation, dtype: int64

In [12]:
bh_df.info()
print('\n')
bus_cats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982535 entries, 0 to 1982534
Data columns (total 11 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   BusinessName            object  
 1   ChainName               object  
 2   BusinessRating          object  
 3   ReviewCount             int64   
 4   previous_review_cnt     float64 
 5   previous_rating         object  
 6   abs_review_diff         float64 
 7   abs_rating_diff         object  
 8   total_review_cnt_delta  float64 
 9   total_bus_rating_delta  object  
 10  CloseDate               category
dtypes: category(1), float64(3), int64(1), object(6)
memory usage: 153.1+ MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126527 entries, 0 to 126526
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   BusinessKey           126527 non-null  int64   
 1   BusinessName          126527 non-null  

In [14]:
bus_cats_df_uq_bus = bus_cats_df.groupby(['BusinessName']).first()
bus_cats_df_uq_bus.shape

(63311, 10)

In [15]:
bus_cats_df_uq_bus_newyork_fl = bus_cats_df_uq_bus.loc[bus_cats_df_uq_bus['StateName'].isin(values=['Florida', 'New York'])] 
bus_cats_df_uq_bus_newyork_fl.shape

(3864, 10)

In [16]:
bh_df_loc_fl_newyork = pd.merge(left=bh_df, right=bus_cats_df_uq_bus_newyork_fl, on='BusinessName', how='inner', suffixes=(None, '_right'))
bh_df_loc_fl_newyork.shape

(123228, 21)

In [17]:
bh_df_loc_fl_newyork.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123228 entries, 0 to 123227
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   BusinessName            123228 non-null  object  
 1   ChainName               123228 non-null  object  
 2   BusinessRating          123228 non-null  object  
 3   ReviewCount             123228 non-null  int64   
 4   previous_review_cnt     119364 non-null  float64 
 5   previous_rating         119364 non-null  object  
 6   abs_review_diff         119364 non-null  float64 
 7   abs_rating_diff         119364 non-null  object  
 8   total_review_cnt_delta  123187 non-null  float64 
 9   total_bus_rating_delta  123187 non-null  object  
 10  CloseDate               123228 non-null  category
 11  BusinessKey             123228 non-null  int64   
 12  ChainName_right         123228 non-null  object  
 13  PaymentLevelName        123228 non-null  object  
 14  Long

In [19]:
bh_df_loc_fl = bh_df_loc_fl_newyork.loc[bh_df_loc_fl_newyork.StateName == 'Florida']
bh_df_loc_newyork = bh_df_loc_fl_newyork.loc[bh_df_loc_fl_newyork.StateName == 'New York']
print(bh_df_loc_fl.shape)
print(bh_df_loc_newyork.shape)

(64712, 21)
(58516, 21)


In [20]:
bh_df_loc_fl.isna().sum()


BusinessName                 0
ChainName                    0
BusinessRating               0
ReviewCount                  0
previous_review_cnt       1981
previous_rating           1981
abs_review_diff           1981
abs_rating_diff           1981
total_review_cnt_delta      25
total_bus_rating_delta      25
CloseDate                    0
BusinessKey                  0
ChainName_right              0
PaymentLevelName             0
Longitude                    0
Latitude                     0
BusinessCategoryName         0
CityName                     0
CountyName                   0
CountryName                  0
StateName                    0
dtype: int64

In [21]:
bh_df_loc_fl_sin_na = bh_df_loc_fl.dropna(axis=0, subset=['previous_review_cnt']).reset_index(drop=True)
bh_df_loc_fl_sin_na.isna().sum()

BusinessName              0
ChainName                 0
BusinessRating            0
ReviewCount               0
previous_review_cnt       0
previous_rating           0
abs_review_diff           0
abs_rating_diff           0
total_review_cnt_delta    0
total_bus_rating_delta    0
CloseDate                 0
BusinessKey               0
ChainName_right           0
PaymentLevelName          0
Longitude                 0
Latitude                  0
BusinessCategoryName      0
CityName                  0
CountyName                0
CountryName               0
StateName                 0
dtype: int64

In [23]:
bh_df_loc_newyork_sin_na = bh_df_loc_newyork.dropna(axis=0, subset=['previous_review_cnt']).reset_index(drop=True)

In [24]:
fl_review_diff_mean = bh_df_loc_fl_sin_na['abs_review_diff'].mean()
newyork_review_diff_mean = bh_df_loc_newyork_sin_na['abs_review_diff'].mean()

mean_diff = fl_review_diff_mean - newyork_review_diff_mean
mean_diff

0.016107898329804374

In [25]:
cm = CompareMeans(DescrStatsW(bh_df_loc_fl_sin_na['abs_review_diff']), DescrStatsW(bh_df_loc_newyork_sin_na['abs_review_diff']))
summary = cm.summary(usevar='unequal')

In [27]:
data_df = pd.DataFrame(data=summary.data)
data_df

,0,1,2,3,4,5,6
0,,coef,std err,t,P>|t|,[0.025,0.975]
1,subset #1,0.0161,0.004,4.093,0.000,0.008,0.024
